In [ ]:
## setup analysis
execfile(r'D:\measuring\analysis\scripts\setup_analysis.py')
from analysis.lib.purification import purify_pq as ppq; reload(ppq)
from analysis.lib.purification import purify_BK as pbk; reload(pbk)
from analysis.lib.fitting import fit, common
from analysis.lib.pq import pq_tools;reload(pq_tools)
from analysis.lib.purification import purify_analysis as pa; reload(pa)
%matplotlib inline

thetas = ['pi/4','pi/5','pi/6','pi/8']


In [ ]:
### define parameters
start = 2762.5e3
window_length = 50e3
hist_binsize = 100

In [ ]:
offsets_on = True

first_shifted = True
first_unshifted = True

for i,theta in enumerate(thetas):
    print 'Processing theta ', theta
    folder_lt3,folder_lt4 = pa.get_folders_from_theta(theta)
    ssro_calib_lt3,ssro_calib_lt4 = pa.get_ssro_calibs()

    Purify = pa.purify_analysis('purification_analysis',folder_lt3,folder_lt4,ssro_calib_lt3,ssro_calib_lt4)

    all_lt3,all_lt4,offsets,offsets_ch1 = Purify.get_tstamps_and_offsets(return_tstamps = True,shifted_days = [])
    print len(all_lt4)
    if np.size(all_lt4):
        if not(offsets_on):
            offsets = np.zeros(np.size(all_lt4))
            offsets_ch1 = np.zeros(np.size(all_lt4))
            
        (h0, b0), (h1, b1) = pa.get_photon_hists_from_tstamps(all_lt4,folder_lt4,start = start,length = window_length, hist_binsize = hist_binsize, offsets = offsets, offsets_ch1 = offsets_ch1)

        if first_shifted:
            ch0_hist_data_unshifted = h0
            ch1_hist_data_unshifted = h1
            first_shifted = False
        else:
            ch0_hist_data_unshifted += h0
            ch1_hist_data_unshifted += h1

    all_lt3,all_lt4,offsets,offsets_ch1 = Purify.get_tstamps_and_offsets(return_tstamps = True,unshifted_days = [])
    print len(all_lt4)
    (h0, b0), (h1, b1) = pa.get_photon_hists_from_tstamps(all_lt4,folder_lt4,start = start,length = window_length, hist_binsize = hist_binsize)
    if np.size(all_lt4):
        if not(offsets_on):
            offsets = np.zeros(np.size(all_lt4))
            offsets_ch1 = np.zeros(np.size(all_lt4))
            
        (h0, b0), (h1, b1) = pa.get_photon_hists_from_tstamps(all_lt4,folder_lt4,start = start,length = window_length, hist_binsize = hist_binsize, offsets = offsets, offsets_ch1 = offsets_ch1)

        if first_unshifted:

            ch0_hist_data_shifted = h0
            ch1_hist_data_shifted = h1
            first_unshifted = False
        else:
            ch0_hist_data_shifted += h0
            ch1_hist_data_shifted += h1

In [ ]:
fig, (ax0) = plt.subplots(figsize=(12,4))
pa.plot_photon_hist(ax0, ch0_hist_data_unshifted, b0,label = 'ch0',log=True)
pa.plot_photon_hist(ax0, ch1_hist_data_unshifted, b1,label = 'ch1',log=True)
ax0.set_title('All data from unshifted days')
plt.show()
plt.close('all')

fig, (ax0) = plt.subplots(figsize=(12,4))
pa.plot_photon_hist(ax0, ch0_hist_data_shifted, b0,label = 'ch0',log=True)
pa.plot_photon_hist(ax0, ch1_hist_data_shifted, b1,label = 'ch1',log=True)
ax0.set_title('All data from shifted days')
plt.show()
plt.close('all')

In [ ]:
def xcorr(a,v):
            
    xaxis = np.arange(len(a)) - len(a)/2
    return xaxis,np.correlate(np.array(a)-np.mean(a),np.array(v)-np.mean(v),'same')


In [ ]:
xaxis,xcorr01_shifted=xcorr(ch0_hist_data_unshifted,ch1_hist_data_unshifted)
xaxis = (b0[1]-b0[0])*xaxis
fig, (ax0) = plt.subplots(figsize=(12,4))
plt.plot(xaxis,xcorr01_shifted)
plt.show()
plt.close('all')
i = np.argmax(xcorr01_shifted)
print 'Ch1 should be shifted by ', xaxis[i]

xaxis,xcorr01_shifted=xcorr(ch0_hist_data_shifted,ch1_hist_data_shifted)
xaxis = (b0[1]-b0[0])*xaxis
fig, (ax0) = plt.subplots(figsize=(12,4))
plt.plot(xaxis,xcorr01_shifted)
plt.show()
plt.close('all')
i = np.argmax(xcorr01_shifted)
print 'Ch1 should be shifted by ', xaxis[i]

total_unshifted_data = ch0_hist_data_unshifted + ch1_hist_data_unshifted
total_shifted_data = ch0_hist_data_shifted + ch1_hist_data_shifted

xaxis,xcorr01_shifted=xcorr(total_unshifted_data,total_shifted_data)
xaxis = (b0[1]-b0[0])*xaxis
fig, (ax0) = plt.subplots(figsize=(12,4))
plt.plot(xaxis,xcorr01_shifted)
plt.show()
plt.close('all')
i = np.argmax(xcorr01_shifted)
print 'Shifted days should be shifted by ', xaxis[i]


In [ ]:
hist_data = []

for theta in thetas:
    print 'Processing theta ', theta
    folder_lt3,folder_lt4 = pa.get_folders_from_theta(theta)
    ssro_calib_lt3,ssro_calib_lt4 = pa.get_ssro_calibs()

    Purify = pa.purify_analysis('purification_analysis',folder_lt3,folder_lt4,ssro_calib_lt3,ssro_calib_lt4)
    all_lt3,all_lt4,offsets,offsets_ch1 = Purify.get_tstamps_and_offsets(return_tstamps = True,unshifted_days = [])
    
    (h0, b0), (h1, b1) = pa.get_photon_hists_from_tstamps(all_lt4,folder_lt4, offsets = offsets, offsets_ch1 = offsets_ch1,start = start,length = window_length, hist_binsize = hist_binsize)
    hist_data.append(h0 + h1)


In [ ]:
def fit_gaussian(ax,x,y,**kw):
    
     ### kw for fitting

    fit_offset = kw.pop('fit_offset',0)
    fit_amplitude = kw.pop('fit_amplitude', 2e4)
    fit_x0 = kw.pop('fit_x0', 2770)
    fit_sigma = kw.pop('fit_sigma', 5)
    fixed = kw.pop('fixed', [])
    show_guess = kw.pop('show_guess', False)
 
    p0,fitfunc,fitfunc_str = common.fit_gauss(fit_offset,fit_amplitude,fit_x0,fit_sigma)

    if show_guess:
        # print decay
        ax.plot(np.linspace(x[0],x[-1],201), fitfunc(np.linspace(x[0],x[-1],201)), ':', lw=2)

    fit_result = fit.fit1d(x,y,None,p0=p0,fitfunc=fitfunc,do_print=True,fixed=fixed,ret=True)

    if isinstance(fit_result, int):
        print "Fit failed!"
       
    return fit_result
   

In [ ]:
start_pos = 2760
end_pos = 2771.0
start_ind = np.argmin(np.abs(b0-start_pos))
end_ind = np.argmin(np.abs(b0-end_pos))

FWHM = 2.83
sigma = FWHM/(2.*np.sqrt(2*np.log(2)))

x0=2769.55

max_pulse_sig = 0.001
for i, theta in enumerate(thetas):
    plt_pos = b0[:-1] + (b0[1]-b0[2])/2.
    fig, (ax0) = plt.subplots(figsize=(12,4))
    plt.plot(plt_pos,(hist_data[i]))
    ax0.set_title(theta)
    
    fit_result = fit_gaussian(ax0,plt_pos[start_ind:end_ind],(hist_data[i][start_ind:end_ind]),fixed = [0,2],fit_x0=x0,fit_sigma = sigma, show_guess = False)

    full_gaussian  = fit_result['fitfunc'](plt_pos)       
    
    plt.plot(plt_pos,full_gaussian)
    diff_sig = hist_data[i] - full_gaussian
    
    for z in range(1,len(plt_pos)):
        relative_signal = np.sum(full_gaussian[-z:])/np.sum(diff_sig[-z:])
        if relative_signal > max_pulse_sig:
            print 'Threshold is at ' ,plt_pos[-z]
            break
    plt.plot(plt_pos,diff_sig)
            
    plt.show()
    plt.close('all')